# Clean ZIMAS / zoning file

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import intake
import utils

In [2]:
catalog = intake.open_catalog("../catalogs/*.yml")

In [3]:
# Default value of display.max_rows is 10 i.e. at max 10 rows will be printed.
# Set it None to display all rows in the dataframe
pd.set_option('display.max_rows', None)

In [4]:
df = catalog.zoning.read()
df = df[['ZONE_CMPLT']].drop_duplicates()

In [5]:
orig = df.copy()

In [55]:
df = orig.copy()

## Parse zoning string

In [ ]:
""" 
split = df.ZONE_CMPLT.str.split('-', expand = True)
split.rename(columns = {0: 'z1', 1: 'z2', 2: 'z3', 3: 'z4', 4: 'z5'}, inplace = True)

# Replace any Nones with blanks, so that later, function can iterate over those as strings
for col in ['z1', 'z2', 'z3', 'z4', 'z5']:
    split[col] = split[col].fillna('')

split.head()

df = pd.concat([df, split], axis = 1)
"""

In [56]:
def parse_zoning(row):
    try:
        z = utils.ZoningInfo(row.ZONE_CMPLT)
        return pd.Series([z.Q, z.T, z.zone_class, z.height_district, z.D, z.overlay], 
                        index = ['Q', 'T', 'zone_class', 'height_district', 'D', 'overlay'])
    except ValueError:
        return pd.Series(['', '', '', '', '', ''], 
                        index = ['Q', 'T', 'zone_class', 'height_district', 'D', 'overlay'])

parsed = df.apply(parse_zoning, axis = 1)

df = pd.concat([df, parsed], axis = 1)

df.head()

,ZONE_CMPLT,Q,T,zone_class,height_district,D,overlay
0,C2-1-SP,False,False,C2,1,False,[SP]
1,C1.5-1-SP,False,False,C1.5,1,False,[SP]
4,[Q]C1.5-1,True,False,C1.5,1,False,None
10,[Q]C1.5-1-RIO,True,False,C1.5,1,False,[RIO]
12,[Q]C1.5-1-O,True,False,C1.5,1,False,[O]


## Zone Class

In [57]:
# Check if valid zone classes are used
valid_zone_class = [
    'A1', 'A2', 'RA',
    'RE', 'RE40', 'RE20', 'RE15', 'RE11', 'RE9',
    'RS', 'R1', 'R1V', 'R1F', 'R1R', 'R1H', 'RU', 'RZ2.5', 'RZ3', 'RZ4', 'RW1',
    'R2', 'RD1.5', 'RD2', 'RD3', 'RD4', 'RD5', 'RD6', 'RMP', 'RW2', 'R3', 'RAS3', 'R4', 'RAS4', 'R5',
    'CR', 'C1', 'C1.5', 'C2', 'C4', 'C5', 'CM',
    'MR1', 'M1', 'MR2', 'M2', 'M3',
    'P', 'PB', 'OS', 'PF', 'SL'
]


df['valid_zone'] = df.zone_class.isin(valid_zone_class)

In [58]:
print(f'# obs with invalid zones: {len(df[df.valid_zone == False])}')
print(f'# of unique invalid zones: {df[df.valid_zone == False].zone_class.nunique()}')

# obs with invalid zones: 137
# of unique invalid zones: 19


In [59]:
# Appears that you can have multiple zones? R3 + P? Is this correct?
# Or, is there a hierarchy, where least restrictive includes all uses of most restrictive, and that spans across residential, agricultural, etc.
df[df.valid_zone == False].zone_class.value_counts()

        80
R3P     10
R4P      9
R1P      8
USC      4
R1R3     4
RAP      3
R1V3     3
R5P      2
RZ5      2
R1V2     2
R1H1     2
R2P      2
CCS      1
R1V1     1
RSP      1
ADP      1
CEC      1
A2P      1
Name: zone_class, dtype: int64

In [60]:
def zone_class(row):
    """
    This function tags the 35 different zone classes based
    on the first letter
    """
    
    open_space_zones = ['OS']
    agricultural_zones = ['A1', 'A2']
    residential_zones = ['RA', 'RE', 'RS', 'R1', 'RU', 'RZ', 'RW1', 
                         'R2', 'RD', 'RMP', 'RW2', 'R3', 'RAS3', 'R4', 'RAS4', 'R5']
    commercial_zones = ['CR', 'C1', 'C1.5', 'C4' ,'C2', 'C5', 'CM']
    manufacturing_zones = ['MR1', 'M1', 'MR2', 'M2', 'M3']
    
    text = row.zone_class
    
    if any(zone in text for zone in open_space_zones):
        return 'open_space'
    if any(zone in text for zone in agricultural_zones):
        return 'agricultural'
    if any(zone in text for zone in residential_zones):
        return 'residential'
    if any(zone in text for zone in commercial_zones):
        return 'commercial'
    if any(zone in text for zone in manufacturing_zones):
        return 'manufacturing'
    if 'PF' in text:
        return 'public_facility'
    if (('P' in text) or ('PB' in text)) and ('PF' not in text):
        return 'parking'
    if 'SL' in text:
        return 'submerged_lands'
    else:
        return ''

df['zone_class_descrip'] = df.apply(zone_class, axis = 1)   

df.zone_class_descrip.value_counts()

residential        871
commercial         567
manufacturing      143
parking            108
                    86
public_facility     67
agricultural        52
open_space          39
submerged_lands      1
Name: zone_class_descrip, dtype: int64

In [61]:
""" 
def reclassify_zone(row):
    if (row.fix_zone == 1) and ('USC' in row.ZONE_CMPLT):
        return row.ZONE_CMPLT
    else:
        return row.zone_class

    
df['overlay'] = df.apply(reclassify_zone, axis = 1)

df[df.fix_zone==1]
"""

" \ndef reclassify_zone(row):\n    if (row.fix_zone == 1) and ('USC' in row.ZONE_CMPLT):\n        return row.ZONE_CMPLT\n    else:\n        return row.zone_class\n\n    \ndf['overlay'] = df.apply(reclassify_zone, axis = 1)\n\ndf[df.fix_zone==1]\n"

## Height District

In [62]:
# Check if valid height districts are used
valid_height_district = [
    '1', '1D', 
    '1L', '1LD', 
    '1VL', '1VLD', 
    '1XL', '1XLD', 
    '1SS', '1SSD', 
    '2', '2D', 
    '3', '3D', 
    '4', '4D', 
]

invalid_height_district = ['1A', '1B']

In [63]:
def valid_heights(row):
    
    text = row.height_district
    
    if any(height in text for height in valid_height_district) and not any(wrong_height in text for wrong_height in invalid_height_district):
        return True
    else: 
        return False

df['valid_height'] = df.apply(valid_heights, axis = 1)

In [64]:
df[df.valid_height == True].height_district.value_counts()

# Still need to grab the correct height district values

1        851
1VL      391
2        245
1XL      194
1L       101
4         38
3          9
EZ1VL      3
2L         2
CSA1       1
2VL        1
Name: height_district, dtype: int64

In [65]:
df[df.valid_height == False].height_district.value_counts()

        80
O        8
RG       4
RIO      1
1B       1
HPOZ     1
1A       1
CPIO     1
CDO      1
Name: height_district, dtype: int64

In [66]:
# There are some getting tagged as height district but still needs further cleaning
def height_district(row):
    """
    This function tags the height district based on
    Table 2 - Height Districts (Height, Stories, FAR & RFAR)
    """
    text = row.height_district
            
    if (('1' in text) or ('1D' in text)) and ('1A' not in text) and ('1B' not in text):
        return '1'
    elif ('1L' in text) or ('1LD' in text):
        return '1L'
    elif ('1VL' in text) or ('1VLD' in text):
        return '1VL'
    elif ('1XL' in text) or ('1XLD' in text):
        return '1XL'
    elif ('1SS' in text) or ('1SSD' in text):
        return '1SS'
    elif ('2' in text) or ('2D' in text):
        return '2'
    elif ('3' in text) or ('3D' in text):
        return '3'
    elif ('4' in text) or ('4D' in text):
        return '4'
    else:
        return ''
    
df['height_district2'] = df.apply(height_district, axis = 1)

In [67]:
df.height_district2.value_counts()

1    1541
2     248
       98
4      38
3       9
Name: height_district2, dtype: int64

In [68]:
df[df.height_district2 ==''].height_district.value_counts()

        80
O        8
RG       4
RIO      1
1B       1
HPOZ     1
1A       1
CPIO     1
CDO      1
Name: height_district, dtype: int64

In [69]:
print(f'# obs with invalid height districts: {len(df[df.valid_height == False])}')
print(f'# of unique invalid height districts: {df[df.valid_height == False].height_district.nunique()}')
df[df.valid_height == False].head()

# obs with invalid height districts: 98
# of unique invalid height districts: 9


,ZONE_CMPLT,Q,T,zone_class,height_district,D,overlay,valid_zone,zone_class_descrip,valid_height,height_district2
70,(WC)TOPANGA-SN-RIO,,,,,,,False,,False,
2000,(F)CM-1-CUGU,,,,,,,False,,False,
2007,(F)CM-1-HPOZ,,,,,,,False,,False,
2009,(F)CM-1-O-HPOZ,,,,,,,False,,False,
2010,(F)R2-1-RIO,,,,,,,False,,False,


In [70]:
""" 
OLD CODE -- revisit if need pieces


# Tag the obs that have additional information in height district column. 
# Move additional info to overlay column
df['fix_height_district'] = df.apply(lambda row: 1 if (row.height_district != '') 
                                     and (row.height_district2 == '') else 0, axis = 1)

df[df.fix_height_district==1]

df['correct_overlay'] = df.apply(lambda row: row.overlay + ', ' + row.height_district if row.fix_height_district == 1 else row.overlay, axis = 1)

df[df.fix_height_district==1]

# There are still some incorrect things that are now moved to correct_overlay (ex: R1R3, CPIO...R1R3 appears in zone_class and correct_overlay)
# It's ok, these incorrect ones just won't map using the supplemental_use_dict


# Replace overlay with correct_height_district
df['overlay'] = df.apply(lambda row: row.correct_overlay if row.fix_height_district == 1 else row.overlay, axis = 1)

# Overwrite height_district with blanks
df['height_district'] = df.apply(lambda row: '' if row.fix_height_district == 1 else row.height_district, axis = 1)


df[df.fix_height_district==1]

df = df.drop(columns = ['fix_height_district', 'correct_overlay'])
"""

" \nOLD CODE -- revisit if need pieces\n\n\n# Tag the obs that have additional information in height district column. \n# Move additional info to overlay column\ndf['fix_height_district'] = df.apply(lambda row: 1 if (row.height_district != '') \n                                     and (row.height_district2 == '') else 0, axis = 1)\n\ndf[df.fix_height_district==1]\n\ndf['correct_overlay'] = df.apply(lambda row: row.overlay + ', ' + row.height_district if row.fix_height_district == 1 else row.overlay, axis = 1)\n\ndf[df.fix_height_district==1]\n\n# There are still some incorrect things that are now moved to correct_overlay (ex: R1R3, CPIO...R1R3 appears in zone_class and correct_overlay)\n# It's ok, these incorrect ones just won't map using the supplemental_use_dict\n\n\n# Replace overlay with correct_height_district\ndf['overlay'] = df.apply(lambda row: row.correct_overlay if row.fix_height_district == 1 else row.overlay, axis = 1)\n\n# Overwrite height_district with blanks\ndf['height

## D Limit

In [71]:
df.D.value_counts()

False    1555
True      299
           80
Name: D, dtype: int64

In [72]:
df[df.D == True].height_district.value_counts()
# Checked, if there is height district followed by a D, those are all tagged as True

2      138
1       61
1VL     39
4       31
1L      22
1XL      6
3        2
Name: height_district, dtype: int64

## Overlay

In [73]:
# Fill in Nones, otherwise cannot do the apply to make the list a string
df.overlay = df.overlay.fillna('')
just_overlay = df[['overlay']]

In [74]:
just_overlay['no_brackets'] = just_overlay['overlay'].apply(', '.join)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [75]:
just_overlay.no_brackets.value_counts()

                 705
RIO              180
O                148
CUGU             141
CDO              100
CPIO              84
HPOZ              73
O, CUGU           54
CDO, RIO          52
K                 45
SN                36
RIO, CUGU         29
HCR               27
H                 26
O, HPOZ           25
CDO, CUGU         21
O, CPIO           18
G, CUGU           17
K, RIO            13
H, HCR            10
RFA               10
H, K               9
SP                 9
CA                 8
O, CDO             8
O, SN              7
O, K               6
RIO, POD           6
HPOZ, CPIO         6
K, CUGU            5
O, CA              5
H, RPD, HCR        4
O, CDO, CUGU       4
K, CPIO            4
H, RIO             4
CDO, SN            4
SN, CPIO           3
G                  3
O, HPOZ, CPIO      3
O, SP              2
K, HPOZ            2
POD                2
O, POD             2
O, HPOZ, CUGU      1
H, RPD             1
CDO, HPOZ          1
H, O, K            1
O, SN, CPIO  

In [76]:
split = just_overlay.no_brackets.str.split(',', expand = True)
split.rename(columns = {0: 'o1', 1: 'o2', 2: 'o3'}, inplace = True)

for col in ['o1', 'o2', 'o3']:
    split[col] = split[col].fillna('')

In [77]:
df = pd.concat([df, split], axis = 1)
df.head()

,ZONE_CMPLT,Q,T,zone_class,height_district,D,overlay,valid_zone,zone_class_descrip,valid_height,height_district2,o1,o2,o3
0,C2-1-SP,False,False,C2,1,False,[SP],True,commercial,True,1,SP,,
1,C1.5-1-SP,False,False,C1.5,1,False,[SP],True,commercial,True,1,SP,,
4,[Q]C1.5-1,True,False,C1.5,1,False,,True,commercial,True,1,,,
10,[Q]C1.5-1-RIO,True,False,C1.5,1,False,[RIO],True,commercial,True,1,RIO,,
12,[Q]C1.5-1-O,True,False,C1.5,1,False,[O],True,commercial,True,1,O,,


In [78]:
# Make a dictionary for supplemental use
supplemental_use_dict = {
    # Supplemental Use found in Table 2 or Zoning Code Article 3
    'O': 'oil_drilling', 
    'S': 'animal_slaughtering', 
    'G': 'surface_mining', 
    'K': 'equinekeeping', 
    'CA': 'commercial_and_artcraft', 
    'MU': 'mixed_use', 
    'FH': 'fence_heights', 
    'SN': 'sign', 
    'HS': 'hillside_standards',
    'RG': 'rear_detached_garage', 
    'RPD': 'residential_planned_development', 
    'POD': 'pedestrian_oriented_district',
    'CDO': 'community_design_overlay',
    'NSO': 'neighborhood_stabilization_ordinance',
    'RFA': 'residential_floor_area',
    'MPR': 'modified_parking_requirement',
    'RIO': 'river_improvement_overlay',
    'HCR': 'hillside_construction_regulation',
    'CPIO': 'community_plan_implementation_overlay',
    'CUGU': 'clean_up_green_up_overlay',
    # Other Zoning Designations found in Zoning Code Article 2, Sec 12.04 Zones - Districts - Symbols.
    'CW': 'central_city_west_specific_plan', 
    'GM': 'glencoe_maxella_specific_plan', 
    'OX': 'oxford_triangle_specific_plan', 
    'PV': 'playa_vista_specific_plan', 
    'WC': 'warner_center_specific_plan', 
    'ADP': 'alameda_district_specific_plan', 
    'CCS': 'century_city_south_studio_zone', 
    'CSA': 'centers_study_area', 
    'PKM': 'park_mile_specific_plan',
    'LAX': 'los_angeles_airport_specific_plan', 
    'HPOZ': 'historic_preservation_overlay_zone', 
    'LASED': 'la_sports_and_entertainment_specific_plan',
    'USC-1A': 'usc_university_park_campus_specific_plan_subarea_1a_zone',
    'USC-1B': 'usc_university_park_campus_specific_plan_subarea_1b_zone',    
    'USC-2': 'usc_university_park_campus_specific_plan_subarea_2_zone',
    'USC-3': 'usc_university_park_campus_specific_plan_subarea_3_zone',
    'PVSP': 'ponte_vista_at_san_pedro_specific_plan'
}

In [79]:
# This is not working as it should -- o2 and o3 aren't getting filled
for col in ['o1', 'o2', 'o3']:
    new_col = f'{col}_descrip'
    df[col] = df[col].str.strip()
    df[new_col] = df[col].map(supplemental_use_dict)
    df[new_col] = df[new_col].fillna('')

In [81]:
df[df.o3 != '']

# Need to clean up where o1 didn't map to anything, and o2 and o3 are filled. Make sure the stuff that isn't overlays is removed.
# Maybe do long first, then make wide again?

,ZONE_CMPLT,Q,T,zone_class,height_district,D,overlay,valid_zone,zone_class_descrip,valid_height,height_district2,o1,o2,o3,o1_descrip,o2_descrip,o3_descrip
798,[Q]C2-1-O-CDO-CUGU,True,False,C2,1,False,"[O, CDO, CUGU]",True,commercial,True,1,O,CDO,CUGU,oil_drilling,community_design_overlay,clean_up_green_up_overlay
2843,C2-1-O-HPOZ-CPIO,False,False,C2,1,False,"[O, HPOZ, CPIO]",True,commercial,True,1,O,HPOZ,CPIO,oil_drilling,historic_preservation_overlay_zone,community_plan_implementation_overlay
4009,C2-2D-O-HPOZ-CPIO,False,False,C2,2,True,"[O, HPOZ, CPIO]",True,commercial,True,2,O,HPOZ,CPIO,oil_drilling,historic_preservation_overlay_zone,community_plan_implementation_overlay
4078,[Q]C2-1VL-O-CDO-CUGU,True,False,C2,1VL,False,"[O, CDO, CUGU]",True,commercial,True,1,O,CDO,CUGU,oil_drilling,community_design_overlay,clean_up_green_up_overlay
4104,C2-1VL-O-HPOZ-CPIO,False,False,C2,1VL,False,"[O, HPOZ, CPIO]",True,commercial,True,1,O,HPOZ,CPIO,oil_drilling,historic_preservation_overlay_zone,community_plan_implementation_overlay
7545,[Q]P-1-O-CDO-CUGU,True,False,P,1,False,"[O, CDO, CUGU]",True,parking,True,1,O,CDO,CUGU,oil_drilling,community_design_overlay,clean_up_green_up_overlay
8084,A1-1-H-RPD-HCR,False,False,A1,1,False,"[H, RPD, HCR]",True,agricultural,True,1,H,RPD,HCR,,residential_planned_development,hillside_construction_regulation
9040,[T][Q]C2-2D-O-SN-CPIO,True,True,C2,2,True,"[O, SN, CPIO]",True,commercial,True,2,O,SN,CPIO,oil_drilling,sign,community_plan_implementation_overlay
34489,R1-1XL-O-HPOZ-CUGU,False,False,R1,1XL,False,"[O, HPOZ, CUGU]",True,residential,True,1,O,HPOZ,CUGU,oil_drilling,historic_preservation_overlay_zone,clean_up_green_up_overlay
50390,RE40-1-H-RPD-HCR,False,False,RE40,1,False,"[H, RPD, HCR]",True,residential,True,1,H,RPD,HCR,,residential_planned_development,hillside_construction_regulation


In [38]:
df[df.ZONE_CMPLT.str.contains('ADP')]

,ZONE_CMPLT,Q,T,zone_class,height_district,D,overlay,valid_zone,zone_class_descrip,valid_height,height_district2,o1,o2,o3,o1_descrip,o2_descrip,o3_descrip
8449,ADP-RIO,False,False,ADP,RIO,False,,False,parking,False,,,,,,,


In [ ]:
# Address the Nones first before converting to int
""" 
Q: Qualified Classification are restrictions to ensure compatibility with surrounding property
T: Tentative Zone Classifications are City Council requirements for public improvements

for col in ['Q', 'T']:
    df[col] = df[col].astype(int)
"""

## Merge back together

In [ ]:
full = catalog.zoning.read()
full.crs = {'init':'epsg:2229'}

In [ ]:
df2 = pd.merge(full, df, on = 'ZONE_CMPLT', how = 'left', validate = 'm:1')